In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.4 MB/s eta 0:00:00


In [ ]:
# Import des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import tqdm
from transformers import ViTImageProcessor, ViTModel
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from PIL import Image
Image.LOAD_TRUNCATED_IMAGES = True
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tqdm import tqdm
import os
import torchvision
import zipfile

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

In [ ]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

In [ ]:
processor.save_pretrained('google/vit-base-patch16-224-in21k')
model.save_pretrained('google/vit-base-patch16-224-in21k')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if True:
  zip_ref = zipfile.ZipFile('/content/drive/MyDrive/trial.zip', 'r')
  zip_ref.extractall()

In [ ]:
class customdataset(Dataset):
    def __init__(self, data_dir, classes_dir, transform=None) -> None:
        super(customdataset, self).__init__()
        self.data_dir = data_dir
        self.classes_dir = classes_dir
        self.transform = transform
        self.images = []
        self.labels = []

        self.class_idx = {class_name: idx for idx, class_name in enumerate(self.classes_dir)}
        self.idx_class = {idx: class_name for idx, class_name in enumerate(self.classes_dir)}

        for class_dir in classes_dir:
            class_images = os.listdir(os.path.join(data_dir, class_dir))
            self.images.extend(class_images)
            self.labels.extend([self.class_idx[class_dir]] * len(class_images))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_name = self.images[index]
        label = self.labels[index]
        image_path = os.path.join(self.data_dir, self.idx_class[label], image_name)
        image = Image.open(image_path).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def get_labels(self):
        return self.labels

    def get_images(self):
        return self.images

    def get_classes(self):
        return self.classes_dir

    def get_class_to_idx(self):
        return {class_name: idx for idx, class_name in enumerate(self.classes_dir)}

    def get_idx_to_class(self):
        return {idx: class_name for idx, class_name in enumerate(self.classes_dir)}

    def get_num_classes(self):
        return len(self.classes_dir)

    def get_num_images(self):
        return len(self.images)

    def get_num_samples(self):
        return len(self.labels)



In [ ]:
data_dir = 'trial'
event_dir = ['Cards', 'Corner', 'Free-Kick', 'Penalty', 'Tackle', 'To_Subtitue']

In [ ]:
thedata = customdataset(data_dir, event_dir, transform=processor)

In [ ]:
# Splitting the data into train and test
TRAIN_PCT = 0.8
train_size = int(TRAIN_PCT * len(thedata))
test_size = len(thedata) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(thedata, [train_size, test_size])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
model.Classifier = nn.Sequential(
    nn.Linear(768, 256),
    nn.ReLU(),
    nn.Linear(256, 6)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
model

ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation(

In [ ]:
# Next we will define the training loop
# We will train the model for 10 epochs
num_epochs = 10
# We will store all the metrics needed to plot the graph in these lists
train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []

# We will use the following to calculate the running training loss
running_loss = 0.0
running_corrects = 0
total = 0

In [ ]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)

In [ ]:
from sklearn.metrics import confusion_matrix

# Define the checkpoint path
checkpoint_path = './checkpoints'

# Move model and data to GPU
model = model.to(DEVICE)

best_val_loss = float('inf')  # Variable to track the best validation loss

# Variables for evaluation metrics
true_labels = []
predicted_labels = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1} of {num_epochs}")
    model.train()  # Set the model to training mode

    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for bidx, (data, targets) in tqdm(enumerate(train_loader)):
        data_batch = data.pixel_values[0].to(DEVICE)

        with torch.set_grad_enabled(True):
            # forward
            outputs = model(data_batch)
            last_hidden_states = outputs.last_hidden_state
            cls_embeddings = last_hidden_states[:, 0, :]
            logits = model.Classifier(cls_embeddings)

            loss = criterion(logits, targets.to(DEVICE))

            # backward
            optimizer.zero_grad()
            loss.backward()
            clip_gradient(model, 1e-1)
            optimizer.step()

        running_loss += loss.item() * data_batch.size(0)
        _, preds = torch.max(logits, 1)
        running_corrects += torch.sum(preds == targets.to(DEVICE).data)
        total_samples += targets.size(0)

        if (bidx + 1) % 100 == 0:
            print(f"Batch {bidx + 1} of {len(train_loader)}")

        # Collect true labels and predicted labels for evaluation
        true_labels.append(targets.tolist())
        predicted_labels.append(preds.tolist())

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects / total_samples
    train_loss.append(epoch_loss)
    train_accuracy.append(epoch_acc)

    print(f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}")

    # Validate the model
    model.eval()  # Set the model to evaluation mode

    val_loss = 0.0
    val_corrects = 0
    val_samples = 0

    with torch.set_grad_enabled(False):
        for tbidx, (data, targets) in tqdm(enumerate(test_loader)):
            data_batch = data.pixel_values[0].to(DEVICE)
            outputs = model(data_batch)
            last_hidden_states = outputs.last_hidden_state
            cls_embeddings = last_hidden_states[:, 0, :]
            logits = model.Classifier(cls_embeddings)

            loss = criterion(logits, targets.to(DEVICE))
            val_loss += loss.item() * data_batch.size(0)

            _, preds = torch.max(logits, 1)
            val_corrects += torch.sum(preds == targets.to(DEVICE).data)
            val_samples += targets.size(0)

            # Collect true labels and predicted labels for evaluation
            true_labels.append(targets.tolist())
            predicted_labels.append(preds.tolist())

    val_loss = val_loss / val_samples
    val_acc = val_corrects / val_samples

    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Compute evaluation metrics
    true_labels2 = np.concatenate(true_labels)
    predicted_labels2 = np.concatenate(predicted_labels)

    confusion_mat = confusion_matrix(true_labels2, predicted_labels2)
    precision = confusion_mat.diagonal() / np.sum(confusion_mat, axis=0)
    recall = confusion_mat.diagonal() / np.sum(confusion_mat, axis=1)
    f1 = 2 * (precision * recall) / (precision + recall)
    mean_precision = np.mean(precision)
    mean_recall = np.mean(recall)
    mean_f1 = np.mean(f1)

    print(f"Mean Precision: {mean_precision:.4f}, Mean Recall: {mean_recall:.4f}, Mean F1: {mean_f1:.4f}")

    # Save the best model checkpoint based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        os.makedirs(checkpoint_path, exist_ok=True)
        checkpoint_file = checkpoint_path + f'model1_epoch{epoch + 1}.pth.tar'
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'train_loss': train_loss,
            'train_accuracy': train_accuracy,
            'val_loss': val_loss,
            'val_acc': val_acc,
            'confusion_matrix': confusion_mat,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'mean_precision': mean_precision,
            'mean_recall': mean_recall,
            'mean_f1': mean_f1
        }, checkpoint_file)

Epoch 1 of 10


100it [03:44,  2.20s/it]

Batch 100 of 413


200it [07:17,  2.06s/it]

Batch 200 of 413


...

Train Loss: 0.1566, Train Acc: 0.9451


104it [02:55,  1.69s/it]


Val Loss: 0.3185, Val Acc: 0.8915
Mean Precision: 0.9012, Mean Recall: 0.9006, Mean F1: 0.9007
Epoch 6 of 10


24it [00:51,  2.15s/it]


KeyboardInterrupt: ignored

In [ ]:
t = []
for i in train_accuracy:
  t.append(i.to('cpu'))

In [ ]:
np.savetxt('test_accuracy', np.array(t))